# Сравнение на Базов и Подобрен Модел

Този notebook сравнява:
- **Базов модел**: `StockTransformer` (само технически features)
- **Подобрен модел**: `StockTransformerWithNews` (технически features + news embeddings)

## Метрики за сравнение:
- MAE (Mean Absolute Error)
- RMSE (Root Mean Squared Error)
- MAPE (Mean Absolute Percentage Error)
- R² (R-squared)
- Directional Accuracy